In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -r requirements.txt

In [1]:
import os
import numpy as np
import tensorflow as tf
import nibabel as nib
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.transform import resize

import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter
from torch.nn.modules.loss import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from utils import DiceLoss
from torchvision import transforms
from utils import test_single_volume

import zipfile

In [2]:
from config import get_config

args = argparse.ArgumentParser()
args.cfg = "configs/swin_tiny_patch4_window7_224_lite.yaml"
args.batch_size = 12
args.cache_mode = 'no'

config = get_config(args)

=> merge config from configs/swin_tiny_patch4_window7_224_lite.yaml


In [12]:
class NiftiDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.filenames = [f.split('.')[0] for f in os.listdir(image_dir) if f.endswith('.nii.gz')]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.filenames[idx] + '.nii.gz')
        label_path_zip = os.path.join(self.label_dir, self.filenames[idx] + '.nii.gz.zip')
        
        # Load image
        image = nib.load(image_path).get_fdata()
        image = torch.tensor(image, dtype=torch.float32)
        
        # Extract and load label
        with zipfile.ZipFile(label_path_zip, 'r') as zip_ref:
            zip_ref.extractall('temp_labels')
        label_path = os.path.join('temp_labels', self.filenames[idx] + '.nii.gz')
        label = nib.load(label_path).get_fdata()
        label = torch.tensor(label, dtype=torch.long)
        
        # Clean up extracted files to save space
        os.remove(label_path)
        
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        sample['case_name'] = self.filenames[idx].strip('\n')
        return sample

In [13]:
image_dir = 'datasets/ULS23_Radboudumc_Bone/images'
label_dir = 'datasets/ULS23_Radboudumc_Bone/labels'

db_train = NiftiDataset(image_dir, label_dir)
print("The length of train set is: {}".format(len(db_train)))
trainloader  = DataLoader(db_train, batch_size=args.batch_size, shuffle=True)

The length of train set is: 744


In [14]:
from networks.vision_transformer import SwinUnet

model = SwinUnet(config, img_size=256, num_classes=2).cuda()
model.load_from(config)

SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.2;num_classes:2
---final upsample expand_first---
pretrained_path:./pretrained_ckpt/swin_tiny_patch4_window7_224.pth
---start load pretrained modle of swin encoder---


In [15]:
model.train()

SwinUnet(
  (swin_unet): SwinTransformerSys(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        dim=96, input_resolution=(56, 56), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=96, input_resolution=(56, 56), num_heads=3, window_size=7, shift_size=0, mlp_ratio=4.0
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=96, window_size=(7, 7), num_heads=3
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
      

In [16]:
ce_loss = CrossEntropyLoss()
dice_loss = DiceLoss(2)
base_lr = 0.05
optimizer = optim.SGD(model.parameters(), lr=base_lr, momentum=0.9, weight_decay=0.0001)

In [17]:
iter_num = 0
max_epoch = 500
max_iterations = max_epoch * len(trainloader)

In [18]:
best_performance = 0.0

In [ ]:
i_batch, sampled_batch = next(enumerate(trainloader))
image_batch, label_batch = sampled_batch['image'], sampled_batch['label']
image_batch, label_batch = image_batch.cuda(), label_batch.cuda()
image_batch

# Ignore

In [ ]:
# Paths to your datasets
data_path = '/content/drive/MyDrive/Data/NGKDStorage/images/ULS23_Part1/ULS23/novel_data/ULS23_Radboudumc_Bone/images'
annotations_path = '/content/drive/MyDrive/Data/NGKDStorage/annotations/ULS23/novel_data/ULS23_Radboudumc_Bone/labels'

In [16]:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
class NiftiDataGenerator(Sequence):
    def __init__(self, data_filenames, annotations_filenames, batch_size, data_path, annotations_path, image_size=(256, 256)):
        self.data_filenames = data_filenames
        self.annotations_filenames = annotations_filenames
        self.batch_size = batch_size
        self.data_path = data_path
        self.annotations_path = annotations_path
        self.image_size = image_size
        self.indexes = self._get_indexes()

    def _get_indexes(self):
        indexes = []
        for i, fname in enumerate(self.data_filenames):
            nii_img = nib.load(os.path.join(self.data_path, fname))
            num_slices = nii_img.shape[2]
            for slice_index in range(num_slices):
                indexes.append((i, slice_index))
        return indexes

    def __len__(self):
        return int(np.ceil(len(self.indexes) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_data = []
        batch_annotations = []

        for i, slice_index in batch_indexes:
            data_filename = self.data_filenames[i]
            annotations_filename = self.annotations_filenames[i]

            data_nii = nib.load(os.path.join(self.data_path, data_filename))
            annotations_nii = nib.load(os.path.join(self.annotations_path, annotations_filename))

            data_img = data_nii.get_fdata()[:, :, slice_index]
            annotations_img = annotations_nii.get_fdata()[:, :, slice_index]

            # Convert grayscale to 3 channels by repeating the slice
            data_slice_3ch = np.repeat(data_img[:, :, np.newaxis], 3, axis=2)

            # Resize images (if necessary)
            data_resized = resize(data_slice_3ch, self.image_size + (3,), anti_aliasing=True)
            annotations_resized = resize(annotations_img, self.image_size, anti_aliasing=True, order=0, preserve_range=True)

            # Remove the unwanted dimension safely
            if data_resized.shape[-1] == 1:
                data_resized = data_resized[..., 0]

            batch_data.append(data_resized)
            batch_annotations.append(annotations_resized)

        return np.array(batch_data), np.array(batch_annotations).astype(np.uint8)

In [ ]:
# Get the list of filenames for data and annotations
data_filenames = [f for f in os.listdir(data_path) if f.endswith('.nii.gz')]
annotations_filenames = [f for f in os.listdir(annotations_path) if f.endswith('.nii.gz')]

In [ ]:
# Sort the lists to ensure they are aligned
data_filenames.sort()
annotations_filenames.sort()

In [ ]:
# Split the filenames into training and validation sets
train_data_filenames, val_data_filenames, train_annotations_filenames, val_annotations_filenames = train_test_split(data_filenames, annotations_filenames, test_size=0.2, random_state=42)

In [ ]:
batch_size = 4  # Adjust based on your GPU memory

# Create the data generators
train_generator = NiftiDataGenerator(
    data_filenames=train_data_filenames,
    annotations_filenames=train_annotations_filenames,
    batch_size=batch_size,
    data_path=data_path,
    annotations_path=annotations_path
)

val_generator = NiftiDataGenerator(
    data_filenames=val_data_filenames,
    annotations_filenames=val_annotations_filenames,
    batch_size=batch_size,
    data_path=data_path,
    annotations_path=annotations_path
)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2

def DeeplabV3Plus(image_size, num_classes):
    # Load a pretrained MobileNetV2 model as a base
    base_model = MobileNetV2(input_shape=[image_size, image_size, 3], include_top=False)

    # Use the activations of these layers
    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]
    base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

    # Create the feature extraction model
    down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)
    down_stack.trainable = False

    # Define the upsampling layers
    up_stack = [
        layers.UpSampling2D(size=(2, 2)),  # 4x4 -> 8x8
        layers.UpSampling2D(size=(2, 2)),  # 8x8 -> 16x16
        layers.UpSampling2D(size=(2, 2)),  # 16x16 -> 32x32
        layers.UpSampling2D(size=(2, 2)),  # 32x32 -> 64x64
        layers.UpSampling2D(size=(2, 2)),  # 64x64 -> 128x128
        layers.UpSampling2D(size=(2, 2)),  # 128x128 -> 256x256
    ]

    inputs = tf.keras.Input(shape=[image_size, image_size, 3])
    x = inputs

    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = layers.Concatenate()
        x = concat([x, skip])

    # This is the last layer of the model
    last = layers.Conv2DTranspose(
        num_classes, 3, strides=2,
        padding='same', activation='softmax')  # Use strides=2 to match the original image size if needed

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
# Define the size of your images and number of classes
image_size = 256  # Size of your input images
num_classes = 2   # Two classes: background and lesion

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback

def dice_score(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(tf.cast(y_true, tf.float32) * tf.cast(y_pred, tf.float32))
    union = tf.reduce_sum(tf.cast(y_true, tf.float32)) + tf.reduce_sum(tf.cast(y_pred, tf.float32))
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice

# Custom callback to print the Dice score after each epoch
class PrintDiceScoreCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch+1}: Dice Score is {logs['dice_score']:.4f} for training and {logs['val_dice_score']:.4f} for validation.")


In [ ]:
# Create the model
model = DeeplabV3Plus(image_size, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[dice_score])


# Print the model summary
model.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_28 (InputLayer)       [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 model_26 (Functional)       [(None, 128, 128, 96),       1841984   ['input_28[0][0]']            
                              (None, 64, 64, 144),                                                
                              (None, 32, 32, 192),                                                
                              (None, 16, 16, 576),                                                
                              (None, 8, 8, 320)]                                                  
                                                                                           

In [ ]:
# Define, compile, and train the model
def train_model():
    model = None  # Declare model variable first
    # Check if a model already exists
    model_path = './my_deeplab_model.h5'
    if os.path.exists(model_path):
        print("Loading existing model.")
        model = tf.keras.models.load_model(model_path, custom_objects={'dice_score': dice_score})
    else:
        print("Creating a new model.")
        model = DeeplabV3Plus(image_size, num_classes)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss='sparse_categorical_crossentropy',
                      metrics=[dice_score])

    # Train the model
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=1,
        steps_per_epoch=len(train_generator),
        validation_steps=len(val_generator),
        callbacks=[PrintDiceScoreCallback()]
    )

    # Save the model after training
    model.save(model_path)

    return history

In [ ]:
# Train the model
history = train_model()

# Plot the Dice score
plt.plot(history.history['dice_score'], label='Training Dice Score')
plt.plot(history.history['val_dice_score'], label='Validation Dice Score')
plt.xlabel('Epoch')
plt.ylabel('Dice Score')
plt.legend()
plt.show()


Creating a new model.
 2264/19040 [==>...........................] - ETA: 10:08:20 - loss: 0.0110 - dice_score: 0.8225